In [29]:
import tensorflow.keras as keras
import keras.applications.resnet50 as resnet50
from tensorflow.keras import backend

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Flatten,ZeroPadding2D, Add
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Activation
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [52]:
import tensorflow.keras as keras

In [59]:
def conv_block(input_data,units):
    units1, units2, units3 = units 
    x=Conv2D(units1,(1,1),padding='same')(input_data)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Conv2D(units2,(3,3),padding='same')(x)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Conv2D(units3,(1,1),padding='same')(x)
    x=BatchNormalization()(x)
    
    shortcut = Conv2D(units3, (1, 1))(input_data)
    shortcut = BatchNormalization()(shortcut)

    x=Add()([x,shortcut ])
    return x

In [63]:
def identity_block(input_data,units):
    units1, units2, units3 = units 
    x=keras.layers.Conv2D(units1,(1,1),padding='same')(input_data)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Activation('relu')(x)
    x=keras.layers.Conv2D(units2,(3,3),padding='same')(x)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Activation('relu')(x)
    x=keras.layers.Conv2D(units3,(3,3),padding='same')(x)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Activation('relu')(x)
    x=keras.layers.Add()([x, input_data])
    
    return x

In [64]:
def buildresnet():
    inputs= keras.Input(shape=(320,480,6))
    x=keras.layers.ZeroPadding2D(padding=(1,1))(inputs)
    x=keras.layers.Conv2D(64,(7,7),padding='same')(x)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Activation('relu')(x)
    x=keras.layers.ZeroPadding2D(padding=(1,1))(x)
    x=keras.layers.MaxPooling2D(3,padding='same')(x)

    x=conv_block(x,[64, 64, 256])
    for i in range(0,2):
        x=identity_block(x,[64, 64, 256])
        
    x=conv_block(x,[128, 128, 512])
    for i in range(0,3):
        x=identity_block(x,[128, 128, 512])
    x=conv_block(x,[256, 256, 1024])
    for i in range(0,5):
        x=identity_block(x,[256, 256, 1024])
    x=conv_block(x,[512, 512, 2048])
    for i in range(0,2):
        x=identity_block(x,[512, 512, 2048])
    x =keras.layers.GlobalAveragePooling2D()(x)
    outputs =keras.layers.Dense(10, activation='softmax')(x)
    resnet = keras.Model(inputs, outputs)
    
    return resnet

In [65]:
resnet = buildresnet()

In [66]:
resnet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 320, 480, 6) 0                                            
__________________________________________________________________________________________________
zero_padding2d_24 (ZeroPadding2 (None, 322, 482, 6)  0           input_13[0][0]                   
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 322, 482, 64) 18880       zero_padding2d_24[0][0]          
__________________________________________________________________________________________________
batch_normalization_66 (BatchNo (None, 322, 482, 64) 256         conv2d_70[0][0]                  
______________________________________________________________________________________________